In [3]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import numpy as np
import pandas as pd
import time
import re
from tqdm import tqdm

# Get Links for Scraping

In [272]:
seed_address_list=[]
seed_page="https://www.immoregion.fr/vente/maison?page="

for i in range(1,10,1):
    current_page=seed_page+str(i)
    page=requests.get(current_page)
    page_content=BeautifulSoup(page.content,"html.parser")
    for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')}):
        if "id-" in link["href"]:
            seed_address_list.append("https://www.immoregion.fr"+link["href"])
    print("{} round".format(i))
    time.sleep(1)
seed_address_list=list(np.unique(seed_address_list))

1 round
2 round
3 round
4 round
5 round
6 round
7 round
8 round
9 round


In [273]:
len(seed_address_list)

180

# Scraping

In [274]:
json_list=[]
for link in tqdm(seed_address_list):
    try:
        #get page content into a var
        page=requests.get(link)
        if page.status_code==200:
            page_content=BeautifulSoup(page.content,"html.parser")
        
        #define the general structure of the data we want to get and squeeze it into a dictionary
            var_names=["ref","constr_year","address","price","price_wo_agency_fee","agency_fee","Honoraires_à_la_charge","area","salon_area","integral_area","furnished","nmbr_of_bathrooms","nmbr_of_bedrooms","nmbr_of_rooms","nmbr_of_sep_toilettes","nmbr_of_shower_rooms","description","kitchen_equiped","open_kitchen","link_addres","scrapping_time","agency","terrasse","parkinglots","cave","pool","Buanderie","furnished_attic","Chimney","Jardin","parkinglots_garage","renovated","energetic_perf","CO2_emission","parquet_flooring","geothermic_heating","solar_panels","gas_heating","wine_cave"]
            json_data={feat:np.nan for feat in var_names}

        #on website there 6 parts containing interesting info, looping through all parts to get the information needed to fill in the dictionary defined above
        for  container in page_content.find_all("div",class_="feature-bloc"):
            if "Détails de la vente" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
                for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
                    if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Prix de vente":
                        json_data["price"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Prix hors honoraires":
                        json_data["price_wo_agency_fee"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Meublé":
                        json_data["furnished"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Honoraires d'agence":
                        json_data["agency_fee"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Honoraires à la charge":
                        json_data["Honoraires_à_la_charge"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text  
                        

            elif "Général" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
                for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
                    if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Surface":
                        json_data["area"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Terrain":
                        json_data["integral_area"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Nombre de pièces":
                        json_data["nmbr_of_rooms"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Nombre de chambres":
                        json_data["nmbr_of_bedrooms"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Année de construction":
                        json_data["constr_year"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Rénové":
                        json_data["renovated"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text

            elif "Intérieur" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
                for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
                    if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Salon":
                        json_data["salon_area"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Salles de bain":
                        json_data["nmbr_of_bathrooms"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Toilettes séparées":
                        json_data["nmbr_of_sep_toilettes"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Salles de douche":
                        json_data["nmbr_of_shower_rooms"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Cuisine ouverte":
                        json_data["open_kitchen"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Cuisine équipée":
                        json_data["kitchen_equiped"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    

            elif "Extérieur" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
                for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
                    if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Terrasse":
                        json_data["terrasse"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Place(s) de parking en extérieur":
                        json_data["parkinglots"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Jardin":
                        json_data["Jardin"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Place(s) de parking dans un garage":
                        json_data["parkinglots_garage"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    
                        
            elif "Autres" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
                for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
                    if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Cave":
                        json_data["cave"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Piscine":
                        json_data["pool"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Buanderie":
                        json_data["Buanderie"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Cheminée":
                        json_data["Chimney"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text        
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Grenier aménagé":
                        json_data["furnished_attic"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Parquet":
                        json_data["parquet_flooring"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Cave à vin":
                        json_data["wine_cave"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            
            elif "Energie / Chauffage" in container.find_all("div",class_="feature-bloc-content-title-name")[0].text:
                for elements in container.find_all("ul",class_="feature-bloc-content-specification")[0].find_all("li"):
                    if elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Diagnostique de performance énergétique (DPE)":
                        json_data["energetic_perf"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Emission de gaz à effet de serre (GES)":
                        json_data["CO2_emission"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Chauffage au gaz":
                        json_data["gas_heating"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Chauffage géothermique":
                        json_data["geothermic_heating"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
                    elif elements.find_all("div",class_="feature-bloc-content-specification-content-name")[0].text=="Panneaux solaires":
                        json_data["solar_panels"]=elements.find_all("div",class_="feature-bloc-content-specification-content-response")[0].text
            
        json_data["ref"]=page_content.find_all("h1",class_="KeyInfoBlockStyle__PdpTitle-sc-1o1h56e-2 hWEtva")[0].find_all("span")[0].text 
        json_data["description"]=page_content.find_all("div",class_="DescriptionStyle__DescriptionContentDropdown-dqxmgt-3 dyQEIL")[0].find_all("div")[0].text
        json_data["link_address"]=link
        json_data["scrapping_time"]=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        json_data["agency"]=page_content.find_all("h3",class_="FormAgencyInfoStyle__AgencyName-sc-10hhmvq-5 jfZIgK")[0].text
        json_data["address"]=page_content.find_all("div",class_="block-localisation-address")[0].text
        json_list.append(json_data)
        time.sleep(2)
    except:
        pass


0%|          | 0/180 [00:00<?, ?it/s]  Détails de la vente
  Général
  Intérieur
  Extérieur
  Autres
  Energie / Chauffage
  1%|          | 1/180 [00:02<07:08,  2.39s/it]  Détails de la vente
  Général
  Energie / Chauffage
  1%|          | 2/180 [00:04<07:03,  2.38s/it]  Détails de la vente
  Général
  Intérieur
  Energie / Chauffage
  2%|▏         | 3/180 [00:07<07:18,  2.48s/it]  Détails de la vente
  Général
  Intérieur
  Energie / Chauffage
  2%|▏         | 4/180 [00:10<07:26,  2.53s/it]  Détails de la vente
  Général
  Intérieur
  Extérieur
  Autres
  Energie / Chauffage
  3%|▎         | 5/180 [00:12<07:37,  2.62s/it]  Détails de la vente
  Général
  Intérieur
  Extérieur
  Energie / Chauffage
  3%|▎         | 6/180 [00:15<07:39,  2.64s/it]  Détails de la vente
  Général
  Intérieur
  Extérieur
  Energie / Chauffage
  4%|▍         | 7/180 [00:18<07:43,  2.68s/it]  Détails de la vente
  Général
  Intérieur
  Extérieur
  Autres
  Energie / Chauffage
  4%|▍         | 8/180 [00:21<0